In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import requests
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter, namedtuple
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from itertools import accumulate, zip_longest, pairwise, tee
from pprint import pprint
from functools import cache
import numpy as np
from enum import Enum, IntEnum
from string import ascii_lowercase, ascii_uppercase
from dataclasses import dataclass, field
import operator
from typing import List, Tuple, Set, Dict
import functools
np.set_printoptions(linewidth=160)

In [46]:
DAY = 5

In [47]:
raw_data, data = get_problem_input(DAY)

In [301]:
test_data = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [302]:
def create_re(str_in):
    return re.compile(f"^({str_in})(.*?)(?:\n\n)", re.DOTALL|re.MULTILINE)

all_headers = ["seeds:",
"seed-to-soil map:",
"soil-to-fertilizer map:",
"fertilizer-to-water map:",
"water-to-light map:",
"light-to-temperature map:",
"temperature-to-humidity map:",
"humidity-to-location map:"]
        
res_in = [create_re(str_in) for str_in in all_headers]

In [344]:
class Range:
    def __init__(self, d_start, s_start, length):
        # inclusive
        self.d_start = d_start
        self.s_start = s_start
        self.d_end = d_start + length - 1
        self.s_end = s_start + length - 1
    
    def __str__(self):
        return f"{self.s_start}-{self.s_end} => {self.d_start}-{self.d_end}"
    
    def __repr__(self):
        return str(self)
    
    def inside(self, x):
        return (x>=self.s_start) and (x<=self.s_end)
    
    def inside_rev(self, x):
        return (x>=self.d_start) and (x<=self.d_end)
    
    def map_s(self, x):
        if self.inside(x):
            return (x-self.s_start) + self.d_start
        
        return x
    
    def split(self, value):
        if not self.inside(value):
            raise ValueError
            
        offset = value-self.s_start
        return Range(self.d_start, self.s_start, offset), Range(self.d_start+offset, self.s_start+offset, self.s_end-value+1)
    
    def map_s_rev(self, x):
        if self.inside_rev(x):
            return (x-self.d_start) + self.s_start
        
        return x
        
    def iterate(self):
        yield from range(self.s_start, self.s_end+1)

In [345]:
class ManyRanges:
    def __init__(self, ranges):
        self.ranges = sorted(ranges, key=lambda r: r.s_start)
        
    def __str__(self):
        return ", ".join(str(r) for r in self.ranges)
    
    def __repr__(self):
        return str(self)
    
    def map_s(self, x):
        pos = -1
        if x < self[0].s_start:
            return pos, x
        
        for i,r in enumerate(self.ranges):
            if x < r.s_start:
                return i-0.5, x
            if r.inside(x):
                return i,(x-r.s_start) + r.d_start
        
        if x > self[-1].s_end:
            return i+0.5, x
        
        return None,x
    
    def split(self, value):
        split_i,_ = self.map_s(value)
        
        print(split_i, _, value)
        print(self.ranges)
        if isinstance(split_i, int):
            print("split")
            new_ranges = self[split_i].split(value)
            del self.ranges[split_i]
            self.ranges.insert(split_i, new_ranges[1])
            self.ranges.insert(split_i, new_ranges[0])
        print(self.ranges)
    
    def __iter__(self):
        yield from self.ranges
        
    def __getitem__(self, key):
        return self.ranges[key]
    
    def collapse(self, downstream):
        print(self)
        print(downstream)
        for source in self:
            pos_start = downstream.map_s(source.s_start)
            pos_end = downstream.map_s(source.s_end)
            print(source.s_start, source.s_end)
            print(pos_start, pos_end)
        print()

In [346]:
seeds, maps = parse_maps(test_data + "\n\n", res_in)
def combine_ranges(ranges_in):
    full_maps = []
    for s,d in itertools.pairwise(ranges_in):
        # s.collapse(d)
        s.split(53)
combine_ranges(maps.values())

0 55 53
[50-97 => 52-99, 98-99 => 50-51]
split
[50-52 => 52-54, 53-97 => 55-99, 98-99 => 50-51]
2 38 53
[0-14 => 39-53, 15-51 => 0-36, 52-53 => 37-38]
split
[0-14 => 39-53, 15-51 => 0-36, 52-52 => 37-37, 53-53 => 38-38]
3 49 53
[0-6 => 42-48, 7-10 => 57-60, 11-52 => 0-41, 53-60 => 49-56]
split
[0-6 => 42-48, 7-10 => 57-60, 11-52 => 0-41, 53-52 => 49-48, 53-60 => 49-56]
1 46 53
[18-24 => 88-94, 25-94 => 18-87]
split
[18-24 => 88-94, 25-52 => 18-45, 53-94 => 46-87]
0 89 53
[45-63 => 81-99, 64-76 => 68-80, 77-99 => 45-67]
split
[45-52 => 81-88, 53-63 => 89-99, 64-76 => 68-80, 77-99 => 45-67]
0 54 53
[0-68 => 1-69, 69-69 => 0-0]
split
[0-52 => 1-53, 53-68 => 54-69, 69-69 => 0-0]


In [261]:
def parse_maps(data_in, res_in):
    output = {}
    seed_list = tuple()
    for res in res_in:
        m = res.search(data_in)
        key, data = m.groups()
        key = key[:-1]
        data = data.strip()
        data = list(map(str.strip, data.split("\n")))
        if key == "seeds":
            seed_list = tuple(map(int,data[0].split(' ')))
        else:
            output[key] = ManyRanges([Range(*tuple(map(int,data_str.split(' ')))) for data_str in data])
    return seed_list, output

In [219]:
ar = list(maps.values())[0]

In [220]:
ar

50-97 => 52-99, 98-99 => 50-51

In [221]:
ar.map_s(98)

50

In [133]:
def calc(data_in,res_in):
    seeds, maps = parse_maps(data_in + "\n\n", res_in)
    minx = None
    
    combine_ranges(maps.values())
    return
    print(seeds)

    for seed in seeds:
        # rely on ordered dict
        current = seed
        print(f"## {current}",end="")
        for key,ranges in maps.items():
            possibles = {rang.map_s(current) for rang in ranges}
            if len(possibles) > 1:
                possibles.remove(current)
                current = possibles.pop()
            print(f" -> {current}",end="")
        
        if minx is None or current < minx:
            minx = current
        print(f"    {minx}")
    return minx

In [116]:
calc(test_data, res_in)

[98-99 => 50-51, 50-97 => 52-99] [15-51 => 0-36, 52-53 => 37-38, 0-14 => 39-53]
[15-51 => 0-36, 52-53 => 37-38, 0-14 => 39-53] [53-60 => 49-56, 11-52 => 0-41, 0-6 => 42-48, 7-10 => 57-60]
[53-60 => 49-56, 11-52 => 0-41, 0-6 => 42-48, 7-10 => 57-60] [18-24 => 88-94, 25-94 => 18-87]
[18-24 => 88-94, 25-94 => 18-87] [77-99 => 45-67, 45-63 => 81-99, 64-76 => 68-80]
[77-99 => 45-67, 45-63 => 81-99, 64-76 => 68-80] [69-69 => 0-0, 0-68 => 1-69]
[69-69 => 0-0, 0-68 => 1-69] [56-92 => 60-96, 93-96 => 56-59]


In [70]:
submit_answer(DAY, 1, Out[69])

251346198
("That's the right answer!  You are one gold star closer to restoring snow "
 'operations. [Continue to Part Two]')


True

In [114]:
def calc2(data_in):
    total_gear_ratio = 0
    array, output_map = parse(data_in)
    char_indices = np.argwhere(np.logical_and(array != '.', np.logical_not(np.char.isdigit(array))))
    for this_index in char_indices:
        neighs = array[neighbors(array,this_index,center=False)]
        this_set = set(map(int, neighs[neighs != '.']))
        if len(this_set) == 2:
            gear_ratio = operator.mul(*[output_map[i] for i in this_set])
            total_gear_ratio += gear_ratio
            # print(gear_ratio, this_set, total_gear_ratio)
    return total_gear_ratio

In [115]:
calc2(test_data)

467835

In [116]:
calc2(data)

86841457

In [117]:
submit_answer(DAY, 2, Out[116])

86841457
("That's the right answer!  You are one gold star closer to restoring snow "
 'operations.You have completed Day 3! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True